In [ ]:
#開始包裝上述爬蟲的功能, 包成一個class 方便之後調用
# 爬取所有頁面資料
import time 
import random
import requests
from bs4 import BeautifulSoup
import os
class books_webcrl:
    def __init__(self):
        pass

    def make_dir(self,path): #偷偷註記 path= './img'....
        if not os.path.exists(path):
            os.makedirs(path)


    def webcrl(self,my_key,page_from,target_page):
        title_list = []
        price_list = []
        img_ink_list = []
        while page_from<=target_page:
            try:
                rst = random.randint(10,25)
                url =  'https://search.books.com.tw/search/query/cat/all/sort/1/v/0/spell/3/ms2/ms2_1/page/'+str(page_from)+'/key/'+my_key
                my_headers = {
                    'Host': 'search.books.com.tw',
                    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.3'  
                }
                response = requests.get(url, headers = my_headers)
                response.text
                soup = BeautifulSoup(response.text , 'html.parser')
                windoes = soup.find_all('table',class_ = "table-searchlist clearfix")
                windows_bs4 = windoes[0]
                boxs = windows_bs4.find_all('tbody')
                boxs # 有很多商品box
                for idx,box in enumerate(boxs):
                    box_a   = box.find('a',target="_blank")
                    box_img = box_a.find('img')
                    img_link = box_img.get('data-srcset')
                    title    = box_a.get('title')

                    boxs_price = box.find_all('td')[2]
                    prices = boxs_price.find('ul', class_="list-nav clearfix")
                    boxs_strong = prices.find_all('strong')
                    price = boxs_strong[-1].text

                    title_list.append(title)
                    price_list.append(price)
                    img_ink_list.append(img_link)
                print(f'======這頁({page_from})內容爬取完成了,休息{rst}秒========')
                page_from = page_from+1

                time.sleep(rst)
            except:
                print('偵測到錯誤')
                time.sleep(5)
                pass
        return title_list , price_list , img_ink_list


In [ ]:
B = books_webcrl()
B.make_dir('./my_imgs')
title_list , price_list , img_ink_list  = B.webcrl('料理',1,3)
#list都有了 開始創建df 做我想要的篩選
df = pd.DataFrame(columns = ['title','price','img_link'])
df['title']    = title_list
df['price']    = price_list
df['img_link'] = img_ink_list
df
#爬完資料,可以存到我們想要的資料夾
if not os.path.exists('./csv_file'):
    os.makedirs('./csv_file')
df.to_csv('./csv_file/my_file.csv')

key_word_1 = '美食'
key_word_2 = '料理'
target_row = []
for row,s in enumerate(df['title']):
    if key_word_1 in s and key_word_2 in s:
        target_row.append(row)
print('與我下的關鍵字有先關的數據位子:',target_row)
target_df = df.iloc[target_row,:]
target_df.to_csv('./csv_file/my_target_file.csv')
target_df